<a href="https://colab.research.google.com/github/PacktPublishing/Machine-Learning-for-Time-Series-with-Python/blob/master/chapter7/Kats.ipynb" target="_parent\"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!MINIMAL=1 pip install kats

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  Using cached kats-0.2.0-py3-none-any.whl.metadata (9.4 kB)
  Using cached numpy-1.21.1.zip (10.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
ERROR: Exception:
Traceback (most recent call last):
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 68, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/commands/install

In [8]:
!pip install "numpy==1.20"

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  Using cached numpy-1.20.0.zip (8.0 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
ERROR: Exception:
Traceback (most recent call last):
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 105, in _run_wrapper
    status = _inner_run()
             ^^^^^^^^^^^^
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/cli/base_command.py", line 96, in _inner_run
    return self.run(options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/cli/req_command.py", line 68, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/root/miniconda3/envs/TimeSeries/lib/python3.12/site-packages/pip/_internal/commands/install.py", line 387, in run
    requirement_set = resolver.resolve(

In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.style.use('seaborn-v0_8-whitegrid')

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "17"

In [10]:
import pandas as pd

owid_covid = pd.read_csv("https://covid.ourworldindata.org/data/owid-covid-data.csv")
owid_covid["date"] = pd.to_datetime(owid_covid["date"])
df = owid_covid[owid_covid.location == "France"].set_index("date", drop=True).resample('D').interpolate(method='linear').reset_index()

IncompleteRead: IncompleteRead(102487109 bytes read, 402322 more expected)

In [ ]:
from kats.models.ensemble.ensemble import EnsembleParams, BaseModelParams
from kats.models.ensemble.kats_ensemble import KatsEnsemble
from kats.models import (
    linear_model,
    quadratic_model
)


model_params = EnsembleParams(
            [
                BaseModelParams("linear", linear_model.LinearModelParams()),
                BaseModelParams("quadratic", quadratic_model.QuadraticModelParams()),
            ]
        )

# create `KatsEnsembleParam` with detailed configurations 
KatsEnsembleParam = {
    "models": model_params,
    "aggregation": "weightedavg",
    "seasonality_length": 30,
    "decomposition_method": "additive",
}

In [ ]:
from kats.consts import TimeSeriesData
TARGET_COL = "new_cases"

df_ts = TimeSeriesData(
    value=df[TARGET_COL], time=df["date"]
)

In [ ]:
m = KatsEnsemble(
    data=df_ts, 
    params=KatsEnsembleParam
).fit()

In [ ]:
m.predict(steps=90).aggregate()

In [ ]:
m.predict(steps=90)
m.aggregate()
m.plot()
plt.ylabel(TARGET_COL)